In [ ]:
import numpy as np
import time
import pandas as pd

from src.fermat_weber.newton import newton_armijo
from src.fermat_weber.weiszfeld import weiszfeld

In [ ]:
def run_instance(m, n, *,
                 method="newton",
                 max_iter=300,
                 domain="unit-square"):
    """
    Corre una instancia aleatoria del problema de Fermat–Weber.
    Retorna dict con f*, iteraciones, tiempo y éxito.
    """

    # ----- generar nodos -----
    if domain == "unit-square":
        A = np.random.rand(m, n)           # [0,1]^n
    elif domain == "normal":
        A = np.random.randn(m, n)          # N(0,1)
    else:
        raise ValueError("Domain inválido.")

    # pesos aleatorios positivos
    w = np.random.rand(m) + 0.1

    t0 = time.perf_counter()

    try:
        if method == "newton":
            res = newton_armijo(A, w, max_iter=max_iter)
        elif method == "weisz":
            res = weiszfeld(A, w, max_iter=max_iter)
        else:
            raise ValueError("Método inválido")

        t1 = time.perf_counter()

        return {
            "f": float(res["f"]),
            "iters": res["k"],
            "time": t1 - t0,
            "success": True
        }
    except Exception as e:
        return {
            "f": np.inf,
            "iters": max_iter,
            "time": time.perf_counter() - t0,
            "success": False
        }

In [ ]:
def benchmark_pair(m, n, N=200):
    """
    Corre N instancias aleatorias comparando Newton vs Weiszfeld.
    Devuelve un DataFrame con estadísticas agregadas.
    """

    stats = []

    for method in ["newton", "weisz"]:
        f_vals = []
        iters = []
        times = []
        failures = 0

        for _ in range(N):
            out = run_instance(m, n, method=method)

            f_vals.append(out["f"])
            iters.append(out["iters"])
            times.append(out["time"])
            failures += 0 if out["success"] else 1

        stats.append({
            "method": method,
            "m": m,
            "n": n,
            "N": N,
            "iter_mean": np.mean(iters),
            "iter_std": np.std(iters),
            "time_mean": np.mean(times),
            "time_std": np.std(times),
            "fail_rate": failures / N,
            "f_mean": np.mean(f_vals)
        })

    return pd.DataFrame(stats)


In [ ]:
results = []

dims = range(2, 11)
for m in [10, 20,  30, 40, 50, 100, 150, 200, 300]:
    for n in dims:
        print(f"Ejecutando benchmark m={m}, n={n} ...")
        df = benchmark_pair(m, n, N=300)
        results.append(df)

results_all = pd.concat(results, ignore_index=True)
results_all


In [ ]:
results_all.to_csv("data")

In [ ]:
def format_results(df):
    df2 = df.copy()
    df2["Iteraciones promedio"] = df2["iter_mean"].round(1)
    df2["Tiempo medio (ms)"] = (1000 * df2["time_mean"]).round(2)
    df2["Tasa de fallo (%)"] = (100 * df2["fail_rate"]).round(1)

    cols = ["method", "m", "n",
            "Iteraciones promedio", "Tiempo medio (ms)", "Tasa de fallo (%)"]

    return df2[cols]

format_results(results_all)
